In [3]:
!pip install json
!pip install torch
!pip install transformers
!pip install faiss-gpu
!pip install numpy

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00


In [4]:
import json
import torch
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np

In [5]:
# Load the tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Sample JSON data
with open('/content/train (1).json', 'r') as f:
    data = json.load(f)  # data should be a list of JSON objects

# Function to embed text using DistilBERT
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Prepare embeddings and associated query types
embeddings = []
query_types = []

for item in data:
    query_embedding = embed_text(item['query'])
    embeddings.append(query_embedding)
    query_types.append(item['question_type'])

# Convert embeddings to a numpy array
embeddings = np.array(embeddings).astype('float32')

# Step 3: Store embeddings in FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # Using L2 distance
index.add(embeddings)  # Add embeddings to the index



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
query_type= []

def classify_query(query):
    query_embedding = embed_text(query).reshape(1, -1)  # Reshape for FAISS
    D, I = index.search(query_embedding.astype('float32'), k=1)  # k=1 for nearest
    closest_index = I[0][0]  # Get the index of the closest match
    query_type.append(query_types[closest_index])
    return query_types[closest_index]


# Example usage
new_query = "Do the TechCrunch article on software companies and the Hacker News article on The Epoch Times both report an increase in revenue related to payment and subscription models, respectively?"

classify_query(new_query)


'comparison_query'

In [7]:
!pip install fastapi uvicorn
!pip install fastapi uvicorn nest_asyncio pyngrok
!npm install -g localtunnel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.4 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [8]:
# Install the required packages if you haven't already
!pip install pyngrok fastapi uvicorn

from pyngrok import ngrok

# Set the Ngrok authentication token
ngrok.set_auth_token("2nRYTwbzoSAssxrsyA6zf3ANWK7_6p6ejgkFWjwBYdNbG8Yqa")

# Open a tunnel to the FastAPI app running on port 8000
public_url = ngrok.connect(8000)

print(f"FastAPI is publicly available at: {public_url}")


FastAPI is publicly available at: NgrokTunnel: "https://a4ba-35-202-130-10.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

# Apply nest_asyncio to allow running uvicorn inside the notebook
nest_asyncio.apply()

# Define the FastAPI app
app = FastAPI()

# Define a Pydantic model for the input query
class QueryRequest(BaseModel):
    query: str

@app.post("/rag_pipeline")
async def rag_pipeline_endpoint(request: QueryRequest):
    query = request.query
    final_answer = rag_pipeline(query)  # Assuming rag_pipeline function is defined elsewhere
    return {"final_answer": final_answer}

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [390]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
